# Load data from Johns Hopkins

In [10]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.core.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import plotly.graph_objects as go
import seaborn as sns
import ipywidgets as widgets

### Data are updated once a day around 23:59 (UTC), except cases_country with hourly update.

In [11]:
confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
death_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
recovered_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
us_confirmed_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv"
us_death_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
country_url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv"

In [12]:
confirmed_df = pd.read_csv(confirmed_url)
confirmed_df.sample(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
199,NaN,Nigeria,9.082000,8.675300,0,0,0,0,0,0,...,162178,162275,162388,162489,162593,162641,162762,162891,162997,163063
198,NaN,Niger,17.607789,8.081666,0,0,0,0,0,0,...,4939,4964,4972,4972,4987,4987,5001,5001,5021,5021
153,NaN,Jamaica,18.109600,-77.297500,0,0,0,0,0,0,...,36670,37128,37458,37747,38227,38514,38848,39237,39543,39967
229,NaN,Solomon Islands,-9.645700,160.156200,0,0,0,0,0,0,...,18,18,18,18,18,18,18,19,19,19
148,NaN,Iran,32.427908,53.688046,0,0,0,0,0,0,...,1823317,1830823,1838803,1846923,1855674,1864984,1875234,1885564,1897314,1908974


### Data collection starts at 22.Jan 2020, up to the current day

In [13]:
confirmed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 441 entries, Province/State to 4/2/21
dtypes: float64(2), int64(437), object(2)
memory usage: 944.1+ KB


In [14]:
confirmed_df.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
count,273.000000,273.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,...,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02,2.740000e+02
mean,20.534804,23.028143,2.032847,2.390511,3.434307,5.229927,7.729927,10.682482,20.357664,22.507299,...,4.556177e+05,4.579944e+05,4.603345e+05,4.624654e+05,4.641822e+05,4.658572e+05,4.679306e+05,4.704234e+05,4.730206e+05,4.753157e+05
std,25.194592,73.596166,26.879101,26.977077,33.585238,46.743494,65.324493,88.014971,215.981285,217.304706,...,2.162368e+06,2.169969e+06,2.177760e+06,2.184772e+06,2.189957e+06,2.195954e+06,2.202630e+06,2.210443e+06,2.219010e+06,2.226723e+06
min,-51.796300,-178.116500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,5.152149,-19.020800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.021000e+03,1.021000e+03,1.021500e+03,1.025250e+03,1.031250e+03,1.043750e+03,1.043750e+03,1.043750e+03,1.044500e+03,1.061250e+03
50%,21.694000,20.939400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.443450e+04,1.443450e+04,1.443450e+04,1.443450e+04,1.443450e+04,1.443450e+04,1.443450e+04,1.494700e+04,1.494700e+04,1.494700e+04
75%,41.112900,84.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.919832e+05,1.936948e+05,1.954990e+05,1.973580e+05,1.983818e+05,1.989432e+05,1.995325e+05,2.000568e+05,2.008212e+05,2.014185e+05
max,71.706900,178.065000,444.000000,444.000000,549.000000,761.000000,1058.000000,1423.000000,3554.000000,3554.000000,...,3.001261e+07,3.008006e+07,3.015737e+07,3.021945e+07,3.026314e+07,3.033256e+07,3.039380e+07,3.046083e+07,3.053987e+07,3.060969e+07


In [15]:
death_df = pd.read_csv(death_url)
death_df.sample(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
239,NaN,Syria,34.802075,38.996815,0,0,0,0,0,0,...,1195,1206,1216,1227,1239,1247,1254,1265,1274,1288
113,NaN,Estonia,58.595300,25.013600,0,0,0,0,0,0,...,823,836,847,860,868,879,896,902,908,922
69,Henan,China,37.895700,114.904200,0,0,0,0,1,1,...,22,22,22,22,22,22,22,22,22,22
123,New Caledonia,France,-20.904305,165.618042,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
14,Victoria,Australia,-37.813600,144.963100,0,0,0,0,0,0,...,820,820,820,820,820,820,820,820,820,820


In [16]:
death_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Columns: 441 entries, Province/State to 4/2/21
dtypes: float64(2), int64(437), object(2)
memory usage: 944.1+ KB


In [17]:
death_df.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
count,273.000000,273.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,...,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000,274.000000
mean,20.534804,23.028143,0.062044,0.065693,0.094891,0.153285,0.204380,0.299270,0.478102,0.485401,...,10014.437956,10056.361314,10100.843066,10136.572993,10160.164234,10189.182482,10231.375912,10276.127737,10319.416058,10356.602190
std,25.194592,73.596166,1.027008,1.028562,1.451957,2.417550,3.142847,4.592341,7.551622,7.552125,...,43234.074990,43406.150157,43584.724894,43731.233417,43812.597722,43906.987548,44077.357752,44258.129471,44434.051980,44581.538159
min,-51.796300,-178.116500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5.152149,-19.020800,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.000000,7.000000,7.250000,7.250000,7.250000,7.250000,7.250000,8.000000,8.000000,8.250000
50%,21.694000,20.939400,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,181.000000,181.000000,181.000000,181.000000,181.000000,181.000000,182.000000,182.000000,182.500000,182.500000
75%,41.112900,84.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2992.000000,2992.000000,3000.750000,3003.000000,3003.000000,3003.000000,3011.750000,3011.750000,3012.500000,3012.500000
max,71.706900,178.065000,17.000000,17.000000,24.000000,40.000000,52.000000,76.000000,125.000000,125.000000,...,545492.000000,547051.000000,548172.000000,548913.000000,549420.000000,550121.000000,550996.000000,552072.000000,553136.000000,554103.000000


In [18]:
recovered_df = pd.read_csv(recovered_url)
recovered_df.sample(5)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
65,Qinghai,China,35.7452,95.9956,0,0,0,0,0,0,...,18,18,18,18,18,18,18,18,18,18
104,French Polynesia,France,-17.6797,149.4068,0,0,0,0,0,0,...,4842,4842,4842,4842,4842,4842,4842,4842,4842,4842
207,NaN,Senegal,14.4974,-14.4524,0,0,0,0,0,0,...,35508,35811,36107,36322,36753,37040,37228,37321,37434,37532
240,British Virgin Islands,United Kingdom,18.4207,-64.6400,0,0,0,0,0,0,...,131,131,131,131,131,131,131,131,131,131
85,NaN,Czechia,49.8175,15.4730,0,0,0,0,0,0,...,1289924,1295205,1299552,1299930,1315571,1342113,1354424,1366461,1371074,1374823


In [19]:
recovered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 259 entries, 0 to 258
Columns: 441 entries, Province/State to 4/2/21
dtypes: float64(2), int64(437), object(2)
memory usage: 892.5+ KB


In [20]:
recovered_df.describe()

,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
count,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,259.000000,...,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02,2.590000e+02
mean,19.102739,28.495821,0.115830,0.123552,0.150579,0.162162,0.216216,0.250965,0.416988,0.490347,...,2.733076e+05,2.745115e+05,2.759192e+05,2.770977e+05,2.783671e+05,2.795426e+05,2.808215e+05,2.822830e+05,2.835970e+05,2.848085e+05
std,24.640880,70.936043,1.743788,1.747697,1.940118,2.005022,2.641273,2.839838,5.005023,5.505010,...,1.071255e+06,1.075752e+06,1.080969e+06,1.085416e+06,1.089808e+06,1.093001e+06,1.098028e+06,1.104350e+06,1.109635e+06,1.113932e+06
min,-51.796300,-178.116500,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.715658,-8.826999,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,8.540000e+02,8.565000e+02,8.565000e+02,8.645000e+02,8.645000e+02,8.645000e+02,8.645000e+02,8.655000e+02,8.655000e+02,8.660000e+02
50%,19.856270,24.603200,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,9.002000e+03,9.330000e+03,9.330000e+03,9.391000e+03,9.454000e+03,9.594000e+03,9.594000e+03,9.668000e+03,9.710000e+03,9.749000e+03
75%,39.018950,90.394950,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.418970e+05,1.427710e+05,1.435825e+05,1.439630e+05,1.451285e+05,1.460390e+05,1.468215e+05,1.474965e+05,1.482600e+05,1.488735e+05
max,71.706900,178.065000,28.000000,28.000000,31.000000,32.000000,42.000000,45.000000,80.000000,88.000000,...,1.123165e+07,1.126464e+07,1.129502e+07,1.132376e+07,1.135599e+07,1.139302e+07,1.143430e+07,1.147468e+07,1.152504e+07,1.156924e+07


In [21]:
us_confirmed_df = pd.read_csv(us_confirmed_url)
us_confirmed_df.sample(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
2621,84047037,US,USA,840,47037.0,Davidson,Tennessee,US,36.170074,-86.786461,...,84612,84787,84917,84917,84917,85234,85331,85496,85672,85672
1067,84021069,US,USA,840,21069.0,Fleming,Kentucky,US,38.367583,-83.696575,...,1108,1109,1112,1112,1114,1116,1117,1119,1120,1120
47,84001095,US,USA,840,1095.0,Marshall,Alabama,US,34.369760,-86.304867,...,11816,11828,11830,11839,11839,11842,11851,11854,11865,11877
92,84002195,US,USA,840,2195.0,Petersburg,Alaska,US,57.139789,-132.954100,...,152,153,153,153,153,153,153,153,159,159
952,84020053,US,USA,840,20053.0,Ellsworth,Kansas,US,38.696777,-98.204628,...,1214,1214,1214,1214,1214,1215,1215,1215,1215,1215


In [22]:
us_death_df = pd.read_csv(us_death_url)
us_death_df.sample(5)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
1429,84027119,US,USA,840,27119.0,Polk,Minnesota,US,47.773725,-96.401346,...,65,65,65,66,66,66,66,66,66,66
1034,84021003,US,USA,840,21003.0,Allen,Kentucky,US,36.751976,-86.194575,...,32,32,32,32,32,32,32,32,33,33
1645,84029510,US,USA,840,29510.0,St. Louis City,Missouri,US,38.635557,-90.243492,...,445,445,445,447,447,448,449,450,452,452
3230,84054087,US,USA,840,54087.0,Roane,West Virginia,US,38.716227,-81.352436,...,8,8,8,8,8,8,8,8,8,8
466,84013113,US,USA,840,13113.0,Fayette,Georgia,US,33.413578,-84.490894,...,154,155,156,156,156,156,156,156,157,157


In [23]:
us_death_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3342 entries, 0 to 3341
Columns: 449 entries, UID to 4/2/21
dtypes: float64(3), int64(440), object(6)
memory usage: 11.4+ MB


In [24]:
us_death_df.describe()

,UID,code3,FIPS,Lat,Long_,Population,1/22/20,1/23/20,1/24/20,1/25/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21
count,3.342000e+03,3342.000000,3332.000000,3342.000000,3342.000000,3.342000e+03,3342.0,3342.0,3342.0,3342.0,...,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000,3342.000000
mean,8.342992e+07,834.494913,33043.078932,36.721617,-88.642045,9.960357e+04,0.0,0.0,0.0,0.0,...,163.223220,163.689707,164.025135,164.246858,164.398564,164.608318,164.870138,165.192101,165.510473,165.799820
std,4.314076e+06,36.487378,18648.808931,9.079322,21.776287,3.241661e+05,0.0,0.0,0.0,0.0,...,650.138376,651.938417,653.198937,654.136463,654.974061,655.583610,656.293257,657.409255,658.745798,660.012395
min,1.600000e+01,16.000000,60.000000,-14.271000,-174.159600,0.000000e+00,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.401811e+07,840.000000,19076.500000,33.896803,-97.803595,9.917250e+03,0.0,0.0,0.0,0.0,...,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000,14.000000
50%,8.402921e+07,840.000000,31012.000000,38.005610,-89.488865,2.489150e+04,0.0,0.0,0.0,0.0,...,41.000000,41.000000,41.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000,42.000000
75%,8.404612e+07,840.000000,47129.500000,41.579255,-82.313398,6.497525e+04,0.0,0.0,0.0,0.0,...,101.000000,101.000000,101.750000,101.750000,101.750000,102.000000,102.000000,102.000000,102.000000,102.000000
max,8.410000e+07,850.000000,99999.000000,69.314792,145.673900,1.003911e+07,0.0,0.0,0.0,0.0,...,22965.000000,23022.000000,23056.000000,23080.000000,23103.000000,23101.000000,23111.000000,23144.000000,23191.000000,23236.000000


In [25]:
country_df = pd.read_csv(country_url)
country_df.sample(5)

,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Incident_Rate,People_Tested,People_Hospitalized,Mortality_Rate,UID,ISO3
33,Central African Republic,2021-04-03 19:20:49,6.6111,20.9394,5245.0,72.0,4957.0,216.0,108.597439,NaN,NaN,1.372736,140,CAF
89,Kazakhstan,2021-04-03 19:20:49,48.0196,66.9237,301818.0,3236.0,271216.0,27366.0,1607.406453,NaN,NaN,1.072169,398,KAZ
157,Slovenia,2021-04-03 19:20:49,46.1512,14.9955,219420.0,4068.0,201446.0,13906.0,10554.457770,NaN,NaN,1.853979,705,SVN
188,West Bank and Gaza,2021-04-03 19:20:49,31.9522,35.2332,248482.0,2681.0,220418.0,25383.0,4870.843703,NaN,NaN,1.078951,275,PSE
42,Cote d'Ivoire,2021-04-03 19:20:49,7.5400,-5.5471,44445.0,250.0,41281.0,2914.0,168.490927,NaN,NaN,0.562493,384,CIV


In [26]:
country_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 192 entries, 0 to 191
Data columns (total 14 columns):
Country_Region         192 non-null object
Last_Update            192 non-null object
Lat                    190 non-null float64
Long_                  190 non-null float64
Confirmed              192 non-null float64
Deaths                 192 non-null float64
Recovered              188 non-null float64
Active                 191 non-null float64
Incident_Rate          190 non-null float64
People_Tested          0 non-null float64
People_Hospitalized    0 non-null float64
Mortality_Rate         192 non-null float64
UID                    192 non-null int64
ISO3                   190 non-null object
dtypes: float64(10), int64(1), object(3)
memory usage: 21.1+ KB


In [27]:
country_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Lat,190.0,19.515850,2.391467e+01,-40.900600,4.643279,17.589241,40.383525,6.496310e+01
Long_,190.0,19.770528,6.202765e+01,-172.104600,-6.706225,20.535638,46.701734,1.780650e+02
Confirmed,192.0,680032.218750,2.641669e+06,1.000000,8410.750000,81264.500000,307879.750000,3.066290e+07
Deaths,192.0,14803.973958,5.275202e+04,0.000000,111.500000,986.500000,6271.750000,5.547170e+05
Recovered,188.0,393506.367021,1.293791e+06,1.000000,5305.750000,48899.000000,222211.250000,1.156924e+07
Active,191.0,123750.586387,5.166782e+05,0.000000,383.500000,5406.000000,33581.500000,4.401462e+06
Incident_Rate,190.0,2726.288197,3.279121e+03,0.673488,158.162413,1338.407710,4803.299634,1.575616e+04
People_Tested,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
People_Hospitalized,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Mortality_Rate,192.0,2.044371,2.379885e+00,0.000000,0.964147,1.661690,2.516426,2.222222e+01


## Conclusion

The first six dataframes contain time series, and we could use them to forecast the evolution of the pandemic. The country_df dataframe contains the last snapshot (last hour) with cumulated data, and we can use it to identify the worst affected countries. 

# Exploratory Data Analysis

### Renaming the df column names to lowercase

In [28]:
country_df.columns = map(str.lower, country_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)

In [29]:
country_df.head()

,country_region,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Afghanistan,2021-04-03 19:20:49,33.93911,67.709953,56595.0,2496.0,51798.0,2301.0,145.382512,NaN,NaN,4.410284,4,AFG
1,Albania,2021-04-03 19:20:49,41.15330,20.168300,126183.0,2256.0,93173.0,30754.0,4384.703593,NaN,NaN,1.787880,8,ALB
2,Algeria,2021-04-03 19:20:49,28.03390,1.659600,117429.0,3099.0,81729.0,32601.0,267.790666,NaN,NaN,2.639041,12,DZA
3,Andorra,2021-04-03 19:20:49,42.50630,1.521800,12174.0,117.0,11428.0,629.0,15756.163852,NaN,NaN,0.961065,20,AND
4,Angola,2021-04-03 19:20:49,-11.20270,17.873900,22467.0,538.0,20867.0,1062.0,68.358841,NaN,NaN,2.394623,24,AGO


### Changing province/state to state and country/region to country

In [30]:
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})

In [31]:
pd.options.display.float_format = '{:,}'.format

In [32]:
confirmed_df_total = confirmed_df.iloc[:, 4:].sum().map('{:,d}'.format)
confirmed_df_total

1/22/20            557
1/23/20            655
1/24/20            941
1/25/20          1,433
1/26/20          2,118
              ...     
3/29/21    127,644,861
3/30/21    128,212,983
3/31/21    128,896,025
4/1/21     129,607,651
4/2/21     130,236,504
Length: 437, dtype: object

In [33]:
recovered_df_total = recovered_df.iloc[:, 4:].sum().map('{:,d}'.format)
recovered_df_total

1/22/20            30
1/23/20            32
1/24/20            39
1/25/20            42
1/26/20            56
              ...    
3/29/21    72,401,542
3/30/21    72,732,773
3/31/21    73,111,302
4/1/21     73,451,626
4/2/21     73,765,414
Length: 437, dtype: object

In [34]:
death_df_total = death_df.iloc[:, 4:].sum().map('{:,d}'.format)
death_df_total

1/22/20           17
1/23/20           18
1/24/20           26
1/25/20           42
1/26/20           56
             ...    
3/29/21    2,791,836
3/30/21    2,803,397
3/31/21    2,815,659
4/1/21     2,827,520
4/2/21     2,837,709
Length: 437, dtype: object

In [46]:
def my_value(number):
    return ("{:,}".format(number))

In [48]:
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())
print('confirmed: ', my_value(confirmed_total))
print('deaths: ', my_value(deaths_total))
print('recovered: ', my_value(recovered_total))
print('active: ', my_value(active_total))

confirmed:  130,566,186
deaths:  2,842,363
recovered:  73,979,197
active:  23,636,362


### Color-coding the confirmed/death/recovered columns

In [49]:
fig = go.FigureWidget( layout=go.Layout() )
def highlight_col(x):
    r = 'background-color: red'
    y = 'background-color: purple'
    g = 'background-color: grey'
    df1 = pd.DataFrame('', index=x.index, columns=x.columns)
    df1.iloc[:, 4] = y
    df1.iloc[:, 5] = r
    df1.iloc[:, 6] = g
    
    return df1

In [50]:
def show_latest_cases(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending= False).head(n).style.apply(highlight_col, axis=None)

## COVID-19 Confirmed/Death/Recovered cases by countries

In [51]:
interact(show_latest_cases, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## Worst hit countries

In [56]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [57]:
# # plotting the 20 worst hit countries

def bubble_chart(n):
    fig = px.scatter(sorted_country_df.head(n), x="country", y="confirmed", size="confirmed", color="country",
               hover_name="country", size_max=60)
    fig.update_layout(
    title=str(n) +" Worst hit countries",
    xaxis_title="Countries",
    yaxis_title="Confirmed Cases",
    width = 700
    )
    fig.show();

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

## Details for a specific country

In [60]:
def plot_cases_of_a_country(country):
    labels = ['confirmed', 'deaths']
    colors = ['blue', 'red']
    mode_size = [6, 8]
    line_size = [4, 5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'] == country].iloc[:,20:]),axis = 0)
        print(i)
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Confirmed Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        paper_bgcolor="lightgrey",
        width = 800,
        
    );
    
    fig.update_yaxes(type="linear")
    fig.show();

In [61]:
interact(plot_cases_of_a_country, country='World')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none' # uncomment this, run cell again - then the graph/figure disappears
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(Text(value='World', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

## 10 worst hit countries - Confirmed cases

In [89]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)

In [62]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "confirmed",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

## 10 worst hit countries - Death cases

In [91]:
sorted_country_df = country_df.sort_values('deaths', ascending= False)

In [92]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "deaths",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

## Worst hit countries - Recovered cases

In [93]:
sorted_country_df = country_df.sort_values('recovered', ascending= False)

In [94]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "recovered",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

## Worst hit countries - Active cases

In [95]:
sorted_country_df = country_df.sort_values('active', ascending= False)

In [96]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "active",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

## Worst hit countries - Mortality cases

In [97]:
sorted_country_df = country_df.sort_values('mortality_rate', ascending= False)

In [98]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "mortality_rate",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

## Worst hit countries - Incident rate

In [99]:
sorted_country_df = country_df.sort_values('incident_rate', ascending= False)

In [100]:
px.bar(
    sorted_country_df.head(10),
    x = "country",
    y = "incident_rate",
    title= "Top 10 worst affected countries", # the axis names
    color_discrete_sequence=["pink"], 
    height=500,
    width=800
)

In [81]:
confirmed_df[confirmed_df.lat.isnull() | confirmed_df.long.isnull()] 

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21


In [82]:
death_df[death_df.lat.isnull() | death_df.long.isnull()] 

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,3/24/21,3/25/21,3/26/21,3/27/21,3/28/21,3/29/21,3/30/21,3/31/21,4/1/21,4/2/21


In [74]:
confirmed_df=confirmed_df.dropna(subset=['long'])
dconfirmed_df=confirmed_df.dropna(subset=['lat'])
death_df=death_df.dropna(subset=['long'])
death_df=death_df.dropna(subset=['lat'])

In [75]:

world_map = folium.Map(location=[11,0], tiles="cartodbpositron", zoom_start=2, max_zoom = 6, min_zoom = 2)


for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'], confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip = "<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>",
        ).add_to(world_map)

world_map
